In [2]:
import pandas as pd
import json
import spacy
from spacy import displacy
import re

2023-10-15 20:31:00.767 Python[93143:2235537] apply_selection_policy_once: avoid use of removable GPUs (via org.python.python:GPUSelectionPolicy->avoidRemovable)


In [3]:
# Define the input CSV file and output JSON file
input_csv_file = 'results.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(input_csv_file)
nlp = spacy.load("en_core_web_sm")

# df.to_excel('full_dataset.xlsx', index=False)
df

,sentence_id,sentence_text
0,1086,I was just enjoying his otherwise good narrati...
1,12241,Because it is apparent that this remote has to...
2,15278,", is definitely a very good strategy game , ye..."
3,19094,"No apparent glitches after 8 months , firmware..."
4,29497,The dangers of Area X are n't immediately appa...
...,...,...
50227,586531181,Shawn 's writing skills were apparent on the s...
50228,586535879,Lionel Richie and Rick James ' prominence at M...
50229,586538436,To prove my point try watching the taped ver...
50230,586557129,It was apparent that the lengh of time it took...


In [4]:
# TODO: Filtered out the "so... what..."
sents = df["sentence_text"]

with_so = set()
without_so = set()

data = []
for s in sents:
    d = {}
    doc = nlp(s)
    adv_idx = {}
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ == "ccomp":
            if token.head.pos_ == "ADJ" and token.head.dep_ == "acomp":
                adj_idx = token.head.idx
                d["adjective"] = token.head.text
                d["verb"] = token.text
                d["sentence"] = s
        if token.pos_ == "ADV":
            adv_idx[token.idx] = token.text
#     raise Exception
    flag = True
    for i in adv_idx:
        try:
            if i < adj_idx:
                if adv_idx[i] == "so":
                    d["tag"] = "with_so"
                    flag = False
                    break
        except:
            continue
    if flag == True:
        d["tag"] = "without_so"
    if d.get("adjective"):
        data.append(d)
        
# Convert the list of dictionaries into a pandas DataFrame
df_washed = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df_washed.to_excel('apparent.xlsx', index=False)

In [6]:
df_new = pd.read_excel('full_dataset.xlsx')
df_old = pd.read_csv('my_file.csv', on_bad_lines='skip')

s_new = set(df_new["sentence"])
s_old = set(df_old["sentence"])

s_difference = s_new-s_old

a_new = set(df_new["adjective"])
a_old = set(df_old["adjective"])

a_difference = a_new-a_old

In [7]:
print(len(s_difference))
print(a_difference)


362
{'nimble'}


In [8]:
diff_df = df_new[df_new['sentence'].isin(list(s_difference))]
diff_df

,adjective,verb,sentence,tag
49,narrow,was,One potential site at Boulder Canyon was bisec...,with_so
63,pleased,visited,Hazel tracked Dot down in May 1989 and visited...,with_so
64,pleased,visited,Hazel tracked Dot down in May 1989 and visited...,with_so
95,dead,wept,But he was dead our master wept < br > \n,without_so
199,thorough,retained,The original kinship terminology of Indo - Eur...,with_so
...,...,...,...,...
21339,strong,forgot,The experience was so strong that Argento neve...,with_so
21437,vigorous,had,This \n attack was so vigorous that Pak army h...,with_so
21594,tactless,broke,Ponsonby reminded Melbourne that the Lamb fami...,with_so
21657,derisory,amounted,They argued that the $ 1000 per day demurrage ...,with_so


In [9]:
diff_df.to_excel('difference.xlsx', index=False)

In [10]:
df_taiqi = pd.read_excel('spike_adj_vb.xlsx')
s_taiqi = set(df_taiqi["Sentence"])

s_difference_taiqi = s_new-s_taiqi

# a_taiqi = set(df_taiqi["adjective"])

# a_difference_taiqi = a_new-a_taiqi
df_taiqi

,Sentence,Adj,Verb,Causal Excess,Causal Licenced Dependent,Non-causal Licensed Dependent,Dummy It,Other Complements,Not a Complement/Not Grammatical,Comments
0,It is unknown how many people attended the eve...,"(5, 8)","(29, 9)",False,False,False,True,False,False,NaN
1,Jose Miguel was very active during the Republi...,"(20, 7)","(145, 8)",False,False,False,False,False,True,NaN
2,"Contrary to common practice, the title of abbo...","(123, 11)","(141, 10)",False,False,False,False,True,False,NaN
3,"Phillips read Elie Wiesel's book ""Night"" about...","(93, 6)","(107, 10)",True,False,False,False,False,False,NaN
4,"It became successful that, driven by with Gil ...","(9, 11)","(58, 4)",False,False,False,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...
9968,"In fact, Schwartz was so isolated from the res...","(24, 9)","(153, 7)",False,False,False,False,False,False,NaN
9969,In the three years during which Representative...,"(158, 6)","(175, 4)",False,False,False,False,False,False,NaN
9970,"It is unclear who committed this crime, but it...","(5, 8)","(17, 10)",False,False,False,False,False,False,NaN
9971,"His brother operated a maiolica workshop, but ...","(55, 6)","(117, 4)",False,False,False,False,False,False,NaN


In [11]:
s_difference_taiqi = s_new-s_taiqi
print(len(s_difference_taiqi))

21095


In [12]:
overlap_taiqi = s_new & s_taiqi
print(len(overlap_taiqi))

12


In [13]:
df_selected = pd.read_excel('selected.xlsx')

In [14]:
def delete_added_sothat(text):
    # Use regular expressions to perform the replacements
    text = re.sub(r' \(so\)', '', text)
    text = re.sub(r' \(that\)', '', text)
    return text

sentences = df_selected["sentences"]
origin_sents = [delete_added_sothat(s) for s in sentences]

print(len(origin_sents))
print(origin_sents)

151
['He was so amazed he forgot to thank me .', 'Marie is amazed that he came up with that so quickly .', 'The decomposition of the corpse was so advanced that the peasants decided to burn it .', 'In fact , it was so technically advanced that the furniture had to be custom designed because no one had ever had data and power connected to every seat on this scale before .', 'One man was so afraid that he camped in the middle of his flock , hoping to evade patrolling cowboys .', 'He was so afraid that rival loyalist inmates wished to kill him inside the prison .', 'Her uncle was so angry that he sent her to live in a Lutheran orphanage , where she spent the rest of her childhood .', "Luka is angry that she cheated on him and put their son 's life in danger .", 'The man was so annoyed that he slept hungry and the next morning he chewed so much of a plant that he took on Shaman powers .', 'Hergé became annoyed that Casterman then sent the book to the printers without his final approval .',

In [15]:
re_df = df_new[~df_new['sentence'].isin(origin_sents)]

In [16]:
len(re_df)

21557

In [17]:
re_df.to_excel('full_dataset_for_rewash.xlsx', index=False)

In [18]:
s = set(df["sentence_text"])

In [19]:
overlap_raw = s & s_taiqi
print(len(overlap_raw))

0


In [20]:
finded = ["amazed", "advanced", "afraid", "angry", "annoyed",
          "apparent", "apprehensive", "astonished", "bad", "bitter", 
          "certain", "clear", "concerned", "cofident", "consistent", 
          "convinced", "delighted", "depressed", "determined", "disappointed", 
          "disappointing", "disgusted", "displeased", "dissatisfied", "ecstatic",
         "elated", "embarrassed", "emphatic", "enthusiastic", "enraged",
         "evident","excited", "fearful", "firm", "fortunate", 
         "frustrated", "furious", "grateful", "guilty",
         "happy", "honored", "horrified", "impressed", "indebted",
         "indignant", "insistent", "irate", "jealous", "outraged",
         "overwhelmed", "overwhelming", "panicked", "pleased",
         "positive", "prevalent", "proud", "resentful", "ridiculous",
         "sad", "satisfied", "scared", "shocked", "strong",
         "stunned", "stunned", "sure", "surprised", "thankful", 
         "thrilled", "unhappy", "unimpressed", "unnerved", "upset",
         "widespread", "worried"]
print(len(finded))

75


In [21]:
df_washed = pd.read_excel('amazon.xlsx')
df_ama = df_washed[~df_washed['adjective'].isin(finded)]
df_no_duplicates = df_ama.drop_duplicates()
df_no_duplicates.to_excel('amazon_unselected.xlsx', index=False)

In [22]:
print(len(df_washed))
print(len(df_no_duplicates))

356418
166032


In [35]:
non_causal = df_taiqi[df_taiqi["Non-causal Licensed Dependent"]]
sentences = non_causal["Sentence"]

adjectives = set()
i = 1
for s in sentences:
    doc = nlp(s)
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ == "ccomp":
            if token.head.pos_ == "ADJ":
                adjectives.add((token.head.text, s))
                break
                
len(adjectives)

118

In [39]:
adj, s = zip(*adjectives)
print(*set(adj), sep="\n")
len(set(adj))

Aware
afraid
aware
insistent
scared
unconvinced
unsure
confident
unaware
sure
concerned
worried
convinced
certain
such
fortunate
cognisant


17